## Excercise (a)- SVD of a $m \times n$ matrix

In [1]:
using LinearAlgebra;
A = rand(3,100); # random matrix

In [2]:
U,S,V = svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
3×3 Matrix{Float64}:
 -0.556551  -0.0687918  -0.82796
 -0.606445  -0.647525    0.46145
 -0.567869   0.758933    0.318662
singular values:
3-element Vector{Float64}:
 9.11939570743726
 3.1346225487114805
 2.31722006078736
Vt factor:
3×100 Matrix{Float64}:
 -0.106418   -0.0861258  -0.0682011  …  -0.111322  -0.168024   -0.0801789
 -0.133051   -0.0472403   0.222821       0.176029  -0.0219669  -0.0812702
  0.0327676  -0.184768    0.0940484     -0.112537   0.0219772  -0.0798568

In [3]:
Ad = A'

100×3 adjoint(::Matrix{Float64}) with eltype Float64:
 0.505939   0.893634   0.25877
 0.8018     0.374628   0.197195
 0.117662   0.0254749  0.952717
 0.81561    0.138528   0.910076
 0.972056   0.875442   0.547972
 0.674332   0.102813   0.772788
 0.25046    0.313095   0.681648
 0.676129   0.711542   0.315182
 0.170029   0.322496   0.191691
 0.486749   0.239425   0.782559
 ⋮                     
 0.0679392  0.900551   0.459445
 0.705423   0.338248   0.958827
 0.775913   0.807308   0.0443962
 0.466802   0.146758   0.457796
 0.410496   0.967266   0.434553
 0.78237    0.958002   0.643016
 0.742956   0.138027   0.912163
 0.815362   0.997327   0.834102
 0.577676   0.522991   0.16291

In [4]:
MMdag = A*A';
MdagM = A'*A;

In [5]:
E,U1 = eigen(MMdag)

E[abs.(E) .< 1E-8] .= zeros(eltype(E))
# sort eigenvalues & eigenvectors in the order of increasing eigenvalues
ids = sortperm(E,rev=true)
E = E[ids]
U1 = U1[:,ids]
S1 = sqrt.(E)


E,V2 = eigen(MdagM)
# remove small values
E[abs.(E) .< 1E-8] .= zeros(eltype(E))
# sort eigenvalues & eigenvectors in the order of increasing eigenvalues
ids = sortperm(E,rev=true)
E = E[ids]
V2 = V2[:,ids]
S2 = sqrt.(E)

100-element Vector{Float64}:
 9.119395707437256
 3.134622548711481
 2.31722006078736
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
size(V2)

(100, 100)

In [7]:
S2 = S2[1:3]; # only 3 largest values are nonzero
V2 = V2[:,1:3]; # selecting all rows and coulmns up to 3

In [8]:
print(norm(S-S1))
println(norm(S-S2))

8.248590975149166e-153.580361673049448e-15


In [9]:
norm(abs.(U)-abs.(U1))

1.990725005259545e-15

In [10]:
norm(abs.(V')-abs.(V2'))

2.7051087550401374e-15

In [11]:
print(size(V'))
println(size(V2'))

(3, 100)(3, 100)


In [40]:
V'

3×100 Matrix{Float64}:
 -0.0968377  -0.0934876  -0.103152   …  -0.0504912  -0.124185    -0.113151
 -0.0757446  -0.174119   -0.135118      -0.0661969  -0.00700964   0.0178864
  0.0365119   0.035401    0.0285371      0.0656819  -0.00527465   0.0909155

In [12]:
abs.(V')  # taking the element wise abs value.

3×100 Matrix{Float64}:
 0.106418   0.0861258  0.0682011  0.115659  …  0.111322  0.168024   0.0801789
 0.133051   0.0472403  0.222821   0.173826     0.176029  0.0219669  0.0812702
 0.0327676  0.184768   0.0940484  0.138684     0.112537  0.0219772  0.0798568

In [13]:


# matrix dimensons
m = 3
nset = (10:10:500)

# number of trials to be averaged
# NOTE: num_avg = 1000 can take some time!
# You can decrease num_avg (but the result 
# could be less accurate)
num_avg = 1000

# computational time
time_svd = zeros(size(nset))
time_eig1 = zeros(size(nset))
time_eig2 = zeros(size(nset))

# changing matrix dimensions
for itn = (1:length(nset))
    n = nset[itn]
    for ita = (1:num_avg)
        M = rand(m,n)
        cput = @elapsed begin
        svd(M)
        end
        time_svd[itn] = time_svd[itn] + cput
        cput = @elapsed begin
        eigen(M*M')
        end
        time_eig1[itn] = time_eig1[itn] + cput
        cput = @elapsed begin
        eigen(M'*M)
        end
        time_eig2[itn] = time_eig2[itn] + cput
    end
end

# take an average
time_svd = time_svd/num_avg;
time_eig1 = time_eig1/num_avg;
time_eig2 = time_eig2/num_avg;

In [15]:
using Plots

plot(nset,time_svd, yaxis=:log, seriestype=:scatter, label = "SVD",
    legend=:bottomright, xlabel = "n", ylabel = "Average CPU Time (s)")
plot!(nset,time_eig1, seriestype=:scatter, label = "Eig1")
plot!(nset,time_eig2, seriestype=:scatter, label = "Eig2")
plot!(nset,time_svd[end]*(nset/maximum(nset)).^2, label = "O(mn^2)")
plot!(nset,maximum(time_eig1).*ones(size(nset)), label = "O(n^3)")
plot!(nset,time_eig2[end]*(nset/maximum(nset)).^3, label = "O(n^3)")

ERROR: LoadError: InitError: 

could not load library "C:\Users\navan\.julia\artifacts\3ac49f33c861f2eb1afea813bec920729529c0e0\bin\avcodec-58.dll"
The specified module could not be found. 
Stacktrace:
  [1] dlopen(s::String, flags::UInt32; throw_error::Bool)
    @ Base.Libc.Libdl .\libdl.jl:117
  [2] dlopen(s::String, flags::UInt32)
    @ Base.Libc.Libdl .\libdl.jl:116
  [3] macro expansion
    @ C:\Users\navan\.julia\packages\JLLWrappers\pG9bm\src\products\library_generators.jl:63 [inlined]
  [4] __init__()
    @ FFMPEG_jll C:\Users\navan\.julia\packages\FFMPEG_jll\wpaSV\src\wrappers\x86_64-w64-mingw32.jl:33
  [5] run_module_init(mod::Module, i::Int64)
    @ Base .\loading.jl:1128
  [6] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base .\loading.jl:1116
  [7] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base .\loading.jl:1061
  [8] _require_search_from_serialized(pkg::Base.PkgId, sourcepath::String, build_id::UInt12

ERROR: LoadError: 

Failed to precompile FFMPEG [c87230d0-a227-11e9-1b43-d7ebe4e7570a] to "C:\\Users\\navan\\.julia\\compiled\\v1.10\\FFMPEG\\jl_AA0B.tmp".
Stacktrace:
  [1] error(s::String)
    @ Base .\error.jl:35
  [2] compilecache(pkg::Base.PkgId, path::String, internal_stderr::IO, internal_stdout::IO, keep_loaded_modules::Bool)
    @ Base .\loading.jl:2462
  [3] compilecache
    @ Base .\loading.jl:2334 [inlined]
  [4] (::Base.var"#968#969"{Base.PkgId})()
    @ Base .\loading.jl:1968
  [5] mkpidlock(f::Base.var"#968#969"{Base.PkgId}, at::String, pid::Int32; kwopts::@Kwargs{stale_age::Int64, wait::Bool})
    @ FileWatching.Pidfile C:\Users\navan\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:93
  [6] #mkpidlock#6
    @ FileWatching.Pidfile C:\Users\navan\.julia\juliaup\julia-1.10.0+0.x64.w64.mingw32\share\julia\stdlib\v1.10\FileWatching\src\pidfile.jl:88 [inlined]
  [7] trymkpidlock(::Function, ::Vararg{Any}; kwargs::@Kwargs{stale_age::Int64})
    @ 

    @ Base .\loading.jl:2216
 [28] top-level scope
    @ stdin:3
in expression starting at C:\Users\navan\.julia\packages\Plots\sxUvK\src\animation.jl:1
in expression starting at C:\Users\navan\.julia\packages\Plots\sxUvK\src\Plots.jl:1
in expression starting at stdin:3


ErrorException: Failed to precompile Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80] to "C:\\Users\\navan\\.julia\\compiled\\v1.10\\Plots\\jl_8F65.tmp".

### Excercise 2

In [18]:
using LinearAlgebra
d = 4
D = 300

300

In [19]:
M1 = rand(D,d,D);
M2 = rand(D,d,d,D);

In [20]:
M1t = reshape(M1,(D,d*D));
SVD = svd(M1t);

In [15]:
U=SVD.U;
V=SVD.Vt;
S=diagm(SVD.S);

Lambda = U*S;
size(Lambda)

(300, 300)

In [17]:
size(V)

(300, 1200)

In [21]:
# checking right isometry
A = V*V';
size(A)

(300, 300)

In [37]:
include("../../22tn-j_CodeRepository/Tensor.jl")

B = reshape(V,(D,d,D));

I = contract(B,3,[2,3],B,3,[2,3])




300×300 Matrix{Float64}:
  1.0          -1.7434e-16   -7.84962e-17  …   6.93889e-18   4.16334e-17
 -1.7434e-16    1.0           5.34078e-16      4.16334e-17   4.11021e-16
 -7.84962e-17   5.34078e-16   1.0              3.1225e-17   -1.62197e-16
  4.33681e-17  -3.52583e-16   2.13458e-15     -1.33574e-16   3.13551e-16
 -6.59195e-17  -6.93889e-17   3.09214e-16     -1.76508e-16   1.45717e-16
  8.67362e-17  -3.20924e-17  -3.59088e-16  …  -1.37043e-16  -5.26922e-17
  2.25514e-17   1.83881e-16   4.39752e-16     -2.08167e-17   8.32667e-17
  3.72966e-17  -3.81639e-17  -4.25007e-17     -1.21431e-16  -1.31839e-16
 -7.89299e-17   2.25514e-17  -1.16226e-16     -1.63064e-16  -2.68882e-17
  2.25514e-17  -5.89806e-17   1.01481e-16      4.51028e-17  -1.52656e-16
  ⋮                                        ⋱                
  4.33681e-17  -2.02963e-16   4.33681e-17     -5.15213e-16  -2.25514e-16
  6.07153e-17   7.89299e-17  -6.245e-17        4.82253e-16   3.55618e-17
  2.60209e-17  -2.77556e-16  -3.81639e

In [26]:
print(size(Lambda))
println(size(B))

(300, 300)

(300, 4, 300)


In [30]:


Mresult = contract(Lambda,2,[2],B,3,[1])

300×4×300 Array{Float64, 3}:
[:, :, 1] =
 0.883972   0.506467  0.675768  0.907037
 0.70773    0.60443   0.64756   0.155138
 0.105917   0.690331  0.974324  0.290212
 0.620896   0.600754  0.642784  0.369959
 0.545555   0.696987  0.879369  0.719093
 0.385478   0.350556  0.676868  0.213748
 0.734221   0.085405  0.161057  0.807613
 0.0868963  0.892549  0.290974  0.602167
 0.988045   0.212295  0.903408  0.893695
 0.805651   0.987226  0.663982  0.15162
 ⋮                              
 0.620792   0.435563  0.528313  0.497556
 0.0441222  0.919755  0.937713  0.973777
 0.694066   0.974322  0.418525  0.211313
 0.833591   0.803891  0.153634  0.427805
 0.826465   0.63538   0.105079  0.158697
 0.827603   0.143863  0.763532  0.214939
 0.091746   0.168663  0.486207  0.0135837
 0.184679   0.176672  0.66795   0.540098
 0.210542   0.518524  0.834149  0.0134685

[:, :, 2] =
 0.681783  0.800512   0.307297    0.428355
 0.320868  0.232597   0.648627    0.110472
 0.12716   0.0360918  0.00532751  0.528833
 0.3

In [32]:
M1

300×4×300 Array{Float64, 3}:
[:, :, 1] =
 0.883972   0.506467  0.675768  0.907037
 0.70773    0.60443   0.64756   0.155138
 0.105917   0.690331  0.974324  0.290212
 0.620896   0.600754  0.642784  0.369959
 0.545555   0.696987  0.879369  0.719093
 0.385478   0.350556  0.676868  0.213748
 0.734221   0.085405  0.161057  0.807613
 0.0868963  0.892549  0.290974  0.602167
 0.988045   0.212295  0.903408  0.893695
 0.805651   0.987226  0.663982  0.15162
 ⋮                              
 0.620792   0.435563  0.528313  0.497556
 0.0441222  0.919755  0.937713  0.973777
 0.694066   0.974322  0.418525  0.211313
 0.833591   0.803891  0.153634  0.427805
 0.826465   0.63538   0.105079  0.158697
 0.827603   0.143863  0.763532  0.214939
 0.091746   0.168663  0.486207  0.0135837
 0.184679   0.176672  0.66795   0.540098
 0.210542   0.518524  0.834149  0.0134685

[:, :, 2] =
 0.681783  0.800512   0.307297    0.428355
 0.320868  0.232597   0.648627    0.110472
 0.12716   0.0360918  0.00532751  0.528833
 0.3

In [44]:
print("Difference between the original M1 & the reconstructed M1:",
    sum(abs.(M1-Mresult)))

Difference between the original M1 & the reconstructed M1:7.226719757106161e-10

In [50]:
# [ii]

M1 = rand(D,d,D);
M1t = reshape(M1,(D*d,D));
SVD = svd(M1t);

U=SVD.U;
V=SVD.Vt;
S=diagm(SVD.S);

Lambda = S*V;
println(size(Lambda))

A = reshape(U,(D,d,D));

I = contract(A,3,[1,2],A,3,[1,2]); # left isometry

Mresult = contract(A,3,[3],Lambda,2,[1])

sum(abs.(M1-Mresult))


(300, 300)


3.295673346589269e-10

In [51]:
# [iii]

M1 = rand(D,d,d,D);
M1t = reshape(M1,(D*d,d*D));
SVD = svd(M1t);

U=SVD.U;
V=SVD.Vt;
S=diagm(SVD.S);

#Lambda = S*V;
#println(size(Lambda))

A = reshape(U,(D,d,d*D));
B = reshape(V,(d*D,d,D));

#I = contract(A,3,[1,2],A,3,[1,2]); # left isometry

Mresult = contract(A,3,[3],S,2,[1])
Mresult = contract(Mresult,3,[3],B,3,[1])


sum(abs.(M1-Mresult))

4.782619606770455e-9

## Excercise 3

In [2]:
using LinearAlgebra
include("../../22tn-j_CodeRepository/Tensor.jl")

updateLeft (generic function with 1 method)

In [16]:
N = 100; # maximum chain length()
Nkeep = 300; # maximal number of states to keep
LSpace,Id = getLocalSpace("Spin",1/2)

H0 = Id*0; # Hamiltonian for only the 1st site
A0 = getIdentity(1,2,Id,2); # 1st leg is dummy leg (vacuum)

# MPS of the GS
A = Array{Any}(undef,1,N);
# Array{Any} - creates an array that can hold any type of data.
# (undef,1,N) - creates a 1D array of N elements currently holding undefined values.

# initialization
Hnow = H0
D,V = eigen((Hnow+Hnow')/2)
A[1] = contract(A0,3,3,V,2,1)
Hprev = diagm(D)

for itN in (2:N)

    global A, Hprev;

    # step [i-ii]
    # spin operator at the current site; to be used for generating
    # the coupling term at the next iteration
    Sprev = updateLeft([],[],A[itN-1],LSpace,3,A[itN-1])
    
    # step [iii]
    # # add new site
    Anow = getIdentity(Hprev,2,Id,2)
    Hnow = updateLeft(Hprev,2,Anow,[],[],Anow)
    # update the Hamiltonian up to the last sites
    # to the enlarged Hilbert space
    
    # step [iv]
    # # spin-spin interaction
    Sprev_x = (Sprev[:,1,:]+Sprev[:,2,:])/sqrt(2); Rx = size(Sprev_x);
    Sprev_y = (Sprev[:,1,:]-Sprev[:,2,:])/sqrt(2)/1im; Ry = size(Sprev_y);
    if length(Rx) == 2; Sprev_x = reshape(Sprev_x,(Rx[1],1,Rx[2])); end;
    if length(Ry) == 2; Sprev_y = reshape(Sprev_y,(Ry[1],1,Ry[2])); end;
    Sx = reshape((LSpace[:,1,:]+LSpace[:,2,:])/sqrt(2),(2,1,2));
    Sy = reshape((LSpace[:,1,:]-LSpace[:,2,:])/sqrt(2)/1im,(2,1,2));
    Hxy = - updateLeft(Sprev_x,3,Anow,Sx,3,Anow) - updateLeft(Sprev_y,3,Anow,Sy,3,Anow)
    
    # step [v]
    Hnow = Hnow+Hxy
    
    # diagonalize the current Hamiltonian
    D,V = eigen((Hnow+Hnow')/2)
    # sort eigenvalues & eigenvectors in the order of increasing
    # eigenvalues
    ids = sortperm(D)
    D = D[ids]
    V = V[:,ids]
    
    # truncation threshold for energy
    Etr = D[min(length(D),Nkeep)]
    oks = (D .< Etr)
    # true: to keep; false: not to keep
    # keep all degenerate states up to tolerance
    
    if itN < N
        A[itN] = contract(Anow,3,3,V[:,oks],2,1)
    else
        A[itN] = contract(Anow,3,3,reshape(V[:,1],(size(V[:,1])[1],1)),2,1) # select GS at the last step
    end
    Hprev = diagm(D[oks])
    
end

In [15]:
Alc = Array{Any}(undef,1,N);

m = A[1]
alpha = size(m)[1]*size(m)[2]
beta = size(m)[3]
mtr = reshape(m,(alpha,beta))
SVD = svd(mtr)
Alc[1] = reshape(SVD.U,(size(m)[1],size(m)[2],size(m)[3]))
Alc[2] = diagm(SVD.S)*SVD.Vt


N = length(A)
for idx in (2:N)
    println(1)
    m = A[idx]
    alpha = size(m)[1]*size(m)[2]
    beta = size(m)[3]
    
    mtr = reshape(m,(alpha,beta))

    SVD = svd(mtr)
    if idx < length(A)
        Alc[idx+1] = diagm(SVD.S)*SVD.Vt
        Anow = reshape(SVD.U,(size(m)[1],size(m)[2],size(m)[3]))
        Alc[idx] = contract(Alc[idx-1],3,[3],Anow,3,[1])
    else
        Alc[idx] = diagm(SVD.S)*SVD.Vt
    end
end

ErrorException: cannot assign a value to imported variable LinearAlgebra.SVD from module Main

In [54]:
for idx in (2:N)
    print(1)
    m = A[idx]
    alpha = size(m)[1]*size(m)[2]
    beta = size(m)[3]
    mtr = reshape(m,(alpha,beta))
    SVD = svd(mtr)
    if idx < length(A)
        Alc[idx+1] = diagm(SVD.S)*SVD.Vt
        Anow = reshape(SVD.U,(size(m)[1],size(m)[2],size(m)[3]))
        print(size(Alc[idx-1]))
        Alc[idx] = contract(Alc[idx-1],3,[3],Anow,3,[1])
    end
end

1(1, 2, 2)1

(1, 2, 2, 3)

ErrorException: ERR: Input tensor A has a different rank from input rankA.

## correct answer

In [31]:
N = 100; # maximum chain length()
Nkeep = 300; # maximal number of states to keep
LSpace,Id = getLocalSpace("Spin",1/2)

H0 = Id*0; # Hamiltonian for only the 1st site
A0 = getIdentity(1,2,Id,2); # 1st leg is dummy leg (vacuum)

# MPS of the GS
A = Array{Any}(undef,1,N);
# Array{Any} - creates an array that can hold any type of data.
# (undef,1,N) - creates a 1D array of N elements currently holding undefined values.

# initialization
Hnow = H0
D,V = eigen((Hnow+Hnow')/2)
A[1] = contract(A0,3,3,V,2,1)
Hprev = diagm(D)

for itN in (2:N)

    global A, Hprev;

    # step [i-ii]
    # spin operator at the current site; to be used for generating
    # the coupling term at the next iteration
    Sprev = updateLeft([],[],A[itN-1],LSpace,3,A[itN-1])
    
    # step [iii]
    # # add new site
    Anow = getIdentity(Hprev,2,Id,2)
    Hnow = updateLeft(Hprev,2,Anow,[],[],Anow)
    # update the Hamiltonian up to the last sites
    # to the enlarged Hilbert space
    
    # step [iv]
    # # spin-spin interaction
    Sprev_x = (Sprev[:,1,:]+Sprev[:,2,:])/sqrt(2); Rx = size(Sprev_x);
    Sprev_y = (Sprev[:,1,:]-Sprev[:,2,:])/sqrt(2)/1im; Ry = size(Sprev_y);
    if length(Rx) == 2; Sprev_x = reshape(Sprev_x,(Rx[1],1,Rx[2])); end;
    if length(Ry) == 2; Sprev_y = reshape(Sprev_y,(Ry[1],1,Ry[2])); end;
    Sx = reshape((LSpace[:,1,:]+LSpace[:,2,:])/sqrt(2),(2,1,2));
    Sy = reshape((LSpace[:,1,:]-LSpace[:,2,:])/sqrt(2)/1im,(2,1,2));
    Hxy = - updateLeft(Sprev_x,3,Anow,Sx,3,Anow) - updateLeft(Sprev_y,3,Anow,Sy,3,Anow)
    
    # step [v]
    Hnow = Hnow+Hxy
    
    # diagonalize the current Hamiltonian
    D,V = eigen((Hnow+Hnow')/2)
    # sort eigenvalues & eigenvectors in the order of increasing
    # eigenvalues
    ids = sortperm(D)
    D = D[ids]
    V = V[:,ids]
    
    # truncation threshold for energy
    Etr = D[min(length(D),Nkeep)]
    oks = (D .< Etr)
    # true: to keep; false: not to keep
    # keep all degenerate states up to tolerance
    
    if itN < N
        A[itN] = contract(Anow,3,3,V[:,oks],2,1)
    else
        A[itN] = contract(Anow,3,3,reshape(V[:,1],(size(V[:,1])[1],1)),2,1) # select GS at the last step
    end
    Hprev = diagm(D[oks])   
end

In [29]:
for itN in (1:N)
    # reshape A[itN] and then perform SVD

    T = A[itN]
    T = reshape(T,(size(T,1)*size(T,2),size(T,3)))
    SVDT = svd(T)

    A[itN] = reshape(SVDT.U,(convert(Int,size(SVDT.U,1)/size(A[itN],2)),size(A[itN],2),size(SVDT.U,2)))
    
    if itN < N
        A[itN+1] = contract(diagm(SVDT.S)*SVDT.Vt,2,2,A[itN+1],3,1)
    else
        global R1

        ### for the last site itN = N. Performing SVD will leaves left overs (S and Vt)
        ## The global variable collects these left overs and add it to M[itN=N](M[end])
        R1 = diagm(SVDT.S)*SVDT.Vt
        println("R1",R1)

    end
end
SVDR1 = svd(R1)
println("U",SVDR1.U)
println("S",SVDR1.S)
println("V",SVDR1.Vt)
##  V' is a single number which serves as the overall phase factor to the MPS.
#  So we can pass over V' to U.
A[end] = contract(A[end],3,3,SVDR1.U*SVDR1.Vt,2,1);

R1ComplexF64[1.0000000000000948 + 0.0im;;]
UComplexF64[1.0 + 0.0im;;]
S[1.0000000000000948]
VComplexF64[1.0 + 0.0im;;]


In [30]:
t1 = A[end]

299×2×1 Array{ComplexF64, 3}:
[:, :, 1] =
   0.00548527+0.0im   0.00275845+0.0im
   -0.0273329+0.0im    0.0107019+0.0im
   -0.0233745+0.0im   -0.0295166+0.0im
    -0.108079+0.0im    0.0763225+0.0im
    0.0680134+0.0im    0.0491957+0.0im
   -0.0330172+0.0im   -0.0103215+0.0im
   -0.0195713+0.0im   -0.0351162+0.0im
    0.0294046+0.0im    0.0199627+0.0im
   0.00248696+0.0im    0.0244853+0.0im
  -0.00155384+0.0im    0.0244131+0.0im
             ⋮       
 -0.000371028+0.0im  -0.00235747+0.0im
   -0.0150814+0.0im   -0.0529581+0.0im
   0.00485935+0.0im   -0.0401515+0.0im
  -0.00736517+0.0im   0.00754271+0.0im
   0.00933824+0.0im   0.00197647+0.0im
   0.00770107+0.0im   -0.0180889+0.0im
     0.018461+0.0im   -0.0333367+0.0im
    0.0176679+0.0im   -0.0161373+0.0im
   -0.0226323+0.0im   -0.0232275+0.0im

 As per the solution [i] of excercise C, we donot require to do any operation on A[end]. SVD of A[end-1] will result and S and Vt which we need contract with A[end]. So the whole block of ``if itN<N`` will get modified.

In [32]:
for itN in (1:N-1)
    # reshape A[itN] and then perform SVD

    T = A[itN]
    T = reshape(T,(size(T,1)*size(T,2),size(T,3)))
    SVDT = svd(T)

    A[itN] = reshape(SVDT.U,(convert(Int,size(SVDT.U,1)/size(A[itN],2)),size(A[itN],2),size(SVDT.U,2)))
    
    A[itN+1] = contract(diagm(SVDT.S)*SVDT.Vt,2,2,A[itN+1],3,1)

end

t2 = A[end]

299×2×1 Array{ComplexF64, 3}:
[:, :, 1] =
   0.00548527+0.0im   0.00275845+0.0im
   -0.0273329+0.0im    0.0107019+0.0im
   -0.0233745+0.0im   -0.0295166+0.0im
    -0.108079+0.0im    0.0763225+0.0im
    0.0680134+0.0im    0.0491957+0.0im
   -0.0330172+0.0im   -0.0103215+0.0im
   -0.0195713+0.0im   -0.0351162+0.0im
    0.0294046+0.0im    0.0199627+0.0im
   0.00248696+0.0im    0.0244853+0.0im
  -0.00155384+0.0im    0.0244131+0.0im
             ⋮       
 -0.000371028+0.0im  -0.00235747+0.0im
   -0.0150814+0.0im   -0.0529581+0.0im
   0.00485935+0.0im   -0.0401515+0.0im
  -0.00736517+0.0im   0.00754271+0.0im
   0.00933824+0.0im   0.00197647+0.0im
   0.00770107+0.0im   -0.0180889+0.0im
     0.018461+0.0im   -0.0333367+0.0im
    0.0176679+0.0im   -0.0161373+0.0im
   -0.0226323+0.0im   -0.0232275+0.0im

In [33]:
t1 == t2

false

In [36]:
for (i,j) in zip(t1,t2)
    println(i,"\t", j)
 
   # println(i==j)

end

0.005485271817545545 + 0.0im	0.005485271817545911 + 0.0im
-0.027332857482144295 + 0.0im	-0.027332857482146884 + 0.0im
-0.023374476752507447 + 0.0im	-0.02337447675250966 + 0.0im
-0.10807920184155102 + 0.0im	-0.10807920184156125 + 0.0im
0.0680134011657483 + 0.0im	0.06801340116575474 + 0.0im
-0.03301719067309029 + 0.0im	-0.033017190673093416 + 0.0im
-0.01957126750518515 + 0.0im	-0.019571267505187003 + 0.0im
0.02940464137217343 + 0.0im	0.029404641372176216 + 0.0im
0.0024869594642058152 + 0.0im	0.0024869594642060507 + 0.0im
-0.0015538388279652984 + 0.0im	-0.0015538388279654454 + 0.0im
-0.030894124730745102 + 0.0im	-0.030894124730748027 + 0.0im
0.04859649257468285 + 0.0im	0.04859649257468745 + 0.0im
0.024206517003788005 + 0.0im	0.024206517003790298 + 0.0im
-0.028877378975202612 + 0.0im	-0.028877378975205346 + 0.0im
-0.010467789575718337 + 0.0im	-0.01046778957571933 + 0.0im
-0.03505641994587798 + 0.0im	-0.035056419945881295 + 0.0im
0.002484574465481078 + 0.0im	0.0024845744654813135 + 0.0im
-0

0.0im
0.007098084526437067 + 0.0im	0.007098084526437739 + 0.0im
0.00396080166062216 + 0.0im	0.003960801660622536 + 0.0im
0.0265489044300343 + 0.0im	0.026548904430036817 + 0.0im
-0.01449659384345878 + 0.0im	-0.014496593843460153 + 0.0im
-0.027427744902195354 + 0.0im	-0.027427744902197952 + 0.0im
-0.00902296473771344 + 0.0im	-0.009022964737714296 + 0.0im
0.01016769192539445 + 0.0im	0.010167691925395413 + 0.0im
0.0001479179291545674 + 0.0im	0.00014791792915458141 + 0.0im
-0.029771404293495833 + 0.0im	-0.02977140429349865 + 0.0im
0.003663365376084453 + 0.0im	0.0036633653760848 + 0.0im
0.0067313553636948525 + 0.0im	0.00673135536369549 + 0.0im
-0.00026283377731165853 + 0.0im	-0.0002628337773116834 + 0.0im
0.0042830041780402344 + 0.0im	0.0042830041780406395 + 0.0im
-0.002055055627651116 + 0.0im	-0.0020550556276513103 + 0.0im
0.05127002116855296 + 0.0im	0.05127002116855782 + 0.0im
0.022188231452803506 + 0.0im	0.022188231452805608 + 0.0im
-0.013046526725809113 + 0.0im	-0.01304652672581035 + 0.0

 + 0.0im	0.09884365198680627 + 0.0im
-0.04971427092773168 + 0.0im	-0.04971427092773639 + 0.0im
-0.018282835306163328 + 0.0im	-0.01828283530616506 + 0.0im
-0.036800702667500886 + 0.0im	-0.03680070266750437 + 0.0im
0.008317501450925815 + 0.0im	0.008317501450926603 + 0.0im
0.07503492747910102 + 0.0im	0.07503492747910813 + 0.0im
-0.0536901404901365 + 0.0im	-0.053690140490141584 + 0.0im
-0.133979733477333 + 0.0im	-0.13397973347734568 + 0.0im
0.09555341519045449 + 0.0im	0.09555341519046354 + 0.0im
-0.029571196792486057 + 0.0im	-0.029571196792488857 + 0.0im
-0.039618103830259314 + 0.0im	-0.03961810383026306 + 0.0im
-0.005032041147852023 + 0.0im	-0.0050320411478525 + 0.0im
0.12380065179686539 + 0.0im	0.12380065179687712 + 0.0im
-0.02711047735256294 + 0.0im	-0.02711047735256551 + 0.0im
0.001445088486031619 + 0.0im	0.0014450884860317558 + 0.0im
0.1210710935081822 + 0.0im	0.12107109350819366 + 0.0im
-0.05812649361552553 + 0.0im	-0.05812649361553103 + 0.0im
0.05802008766321699 + 0.0im	0.0580200876

im	0.011322321711440472 + 0.0im
-0.012792132696855295 + 0.0im	-0.012792132696856506 + 0.0im
-0.014240647903061602 + 0.0im	-0.014240647903062951 + 0.0im
0.0033135026443032725 + 0.0im	0.0033135026443035865 + 0.0im
-0.0016391407960490898 + 0.0im	-0.001639140796049245 + 0.0im
0.10952683502105692 + 0.0im	0.10952683502106729 + 0.0im
0.04574202794469137 + 0.0im	0.04574202794469571 + 0.0im
-0.022131179259450767 + 0.0im	-0.022131179259452863 + 0.0im
-0.032615941481574484 + 0.0im	-0.03261594148157758 + 0.0im
-0.0003710278127734277 + 0.0im	-0.00037102781277346286 + 0.0im
-0.01508139005836913 + 0.0im	-0.01508139005837056 + 0.0im
0.004859347074188688 + 0.0im	0.004859347074189149 + 0.0im
-0.0073651664825301185 + 0.0im	-0.007365166482530816 + 0.0im
0.009338244440098193 + 0.0im	0.009338244440099078 + 0.0im
0.007701069823253642 + 0.0im	0.007701069823254371 + 0.0im
0.01846096915837526 + 0.0im	0.01846096915837701 + 0.0im
0.017667908832259435 + 0.0im	0.01766790883226111 + 0.0im
-0.022632269497096198 + 0.0


-0.03619025441801644 + 0.0im	-0.03619025441801987 + 0.0im
-0.04781423630577104 + 0.0im	-0.047814236305775575 + 0.0im
0.0015950545953039269 + 0.0im	0.001595054595304078 + 0.0im
-0.01467103118338524 + 0.0im	-0.01467103118338663 + 0.0im
-0.07434995060846897 + 0.0im	-0.074349950608476 + 0.0im
0.009728381865273611 + 0.0im	0.009728381865274534 + 0.0im
0.06419497495108216 + 0.0im	0.06419497495108825 + 0.0im
0.007690219640182986 + 0.0im	0.007690219640183715 + 0.0im
-0.01592575974855629 + 0.0im	-0.0159257597485578 + 0.0im
0.030116522981334232 + 0.0im	0.030116522981337084 + 0.0im
0.010584436214163943 + 0.0im	0.010584436214164945 + 0.0im
-0.017168358203466207 + 0.0im	-0.01716835820346783 + 0.0im
-0.05578176222527105 + 0.0im	-0.05578176222527633 + 0.0im
-0.029760274289140503 + 0.0im	-0.029760274289143324 + 0.0im
0.02733526655930405 + 0.0im	0.027335266559306643 + 0.0im
0.029664665683571474 + 0.0im	0.029664665683574284 + 0.0im
-0.03987804439767615 + 0.0im	-0.03987804439767993 + 0.0im
-0.00831805550

0.0im	-0.009453116373723175 + 0.0im
-0.0030785056018923524 + 0.0im	-0.003078505601892644 + 0.0im
-0.02031340448694044 + 0.0im	-0.02031340448694236 + 0.0im
-0.019936784005170845 + 0.0im	-0.019936784005172733 + 0.0im
0.02567411538536796 + 0.0im	0.025674115385370393 + 0.0im
0.0023552846123789865 + 0.0im	0.00235528461237921 + 0.0im
-0.02495886742543936 + 0.0im	-0.024958867425441723 + 0.0im
0.005068825186221465 + 0.0im	0.005068825186221946 + 0.0im
0.008488988173266956 + 0.0im	0.008488988173267759 + 0.0im
-0.00865020389636925 + 0.0im	-0.00865020389637007 + 0.0im
-0.015092227806708512 + 0.0im	-0.015092227806709942 + 0.0im
-0.01890360524817009 + 0.0im	-0.01890360524817188 + 0.0im
0.010401853192980304 + 0.0im	0.01040185319298129 + 0.0im
0.03323931546187374 + 0.0im	0.03323931546187689 + 0.0im
-0.005040699645423859 + 0.0im	-0.005040699645424336 + 0.0im
0.005050875203201177 + 0.0im	0.005050875203201656 + 0.0im
-0.010175246873744936 + 0.0im	-0.010175246873745901 + 0.0im
0.037211909106833044 + 0.0im

##### While comparing the individual values of `t1` and `t2`, we can see there is slight difference in the values.


In [41]:
#[iv] bond canonical form

id = 50

for itN in (length(A):-1:id+1)
    T = A[itN]
    T = reshape(T,(size(T,1),size(T,2)*size(T,3)))
    SVDT = svd(T)

    A[itN] = reshape(SVDT.Vt,(size(SVDT.V,2),size(A[itN],2),convert(Int,size(SVDT.V,1)/size(A[itN],2))))

    if itN > (id+1)

        A[itN-1] = contract(A[itN-1],3,3,SVDT.U*diagm(SVDT.S),2,1)

    else
        global R2
        # R2: tensor which is the leftover after transforming the right
        # part. It will be contracted with the M tensor at the isometry
        # center.
        R2 = SVDT.U*diagm(SVDT.S)

    end
end
A[id] = contract(A[id],3,3,R2,2,1);
    


299×2×299 Array{ComplexF64, 3}:
[:, :, 1] =
   0.0571721+0.0im   -0.0309075+0.0im
 -0.00333905+0.0im     0.103889+0.0im
   0.0477114+0.0im    -0.091865+0.0im
  -0.0372353+0.0im   0.00801634+0.0im
    0.103803+0.0im     0.013409+0.0im
 -0.00270917+0.0im   -0.0106662+0.0im
   0.0440382+0.0im  -0.00973497+0.0im
   0.0409612+0.0im  -0.00771954+0.0im
   0.0258901+0.0im   -0.0554417+0.0im
   0.0354978+0.0im   0.00668246+0.0im
            ⋮       
 -0.00706623+0.0im    0.0094373+0.0im
   -0.014022+0.0im    0.0143111+0.0im
  -0.0068953+0.0im   -0.0253422+0.0im
  -0.0122522+0.0im   -0.0037587+0.0im
 -0.00739977+0.0im   0.00630842+0.0im
   0.0121385+0.0im     0.017091+0.0im
  0.00483971+0.0im   0.00867397+0.0im
   0.0387798+0.0im    0.0207834+0.0im
   0.0339078+0.0im     -0.02279+0.0im

[:, :, 2] =
   0.0164711+0.0im    0.0205604+0.0im
  -0.0895141+0.0im    0.0204644+0.0im
   0.0569399+0.0im   -0.0117486+0.0im
 -0.00147439+0.0im     -0.01584+0.0im
  0.00127807+0.0im   0.00327743+0.0im
  0.002330

# Excercise - 4

Mixed canonical or Site Canonical MPS have the following form:

![Alt text](image.png)

Mathematically we can write the MPS as:

$\begin{equation}|\Psi\rangle = |\phi_{\beta}\rangle_{l+1}|\sigma_l\rangle|\psi_{\alpha}\rangle_{l+1} M^{\alpha\sigma_{l}\beta}\end{equation}
$

or 

$$|\Psi\rangle = |\sigma_l\rangle\left[A^{\sigma_1},A^{\sigma_2},..A^{\sigma_{l-1}}\right]^{1}_{\alpha}\left[B^{\sigma_{l+1}},B^{\sigma_{l+2}},..B^{\sigma_{L}}\right]^{1}_{\beta}M^{\alpha\sigma_{l}\beta}$$

The entanglement entropy is given by $\sum_{w}(-w\log w)$, where $w$ is the eigenvalue of a reduced density matrix. The reduced density matrix $\rho_{A}$ can be obtained by:

$$Tr_{B}|\Psi\rangle\langle\Psi| = M^{\alpha\sigma_{l}\beta} M^{\dagger}_{\beta^{'}\sigma_{l}^{'}\alpha^{'}}$$

Pictorially we can represent it as:

![Alt text](image-1.png)

Or we can contract the other legs, It corresponds to tracing over subsystem A:

![Alt text](image-2.png)

### Note
- In the solution given they have grouped the indices $\alpha$ and $\sigma_l$.
- I think they are discussing it with respect to the site $l$.
- In the lecture notes, they have represented the BC-MPS as a single site. I think its some sort of Short hand representation.
- Eventhough entropy remains same if we use $\rho_A$ or $\rho_B$,they have different costs.

Let the bond Dimension of $\alpha,\beta,\sigma_l = D,D,d$. Then the cost is $\mathcal{O}(d^{2}D^{3})$ for $\rho_{A}$ and $\mathcal{O}(dD^{3})$ for $\rho_{B}$.
### Note
- `The cost is given differnt in the solution.`

### Bringing the MPS to RC form



In [28]:
using LinearAlgebra
include("../../22tn-j_CodeRepository/Tensor.jl")

updateLeft (generic function with 1 method)

In [31]:
N = 100; # maximum chain length()
Nkeep = 300; # maximal number of states to keep
LSpace,Id = getLocalSpace("Spin",1/2)

H0 = Id*0; # Hamiltonian for only the 1st site
A0 = getIdentity(1,2,Id,2); # 1st leg is dummy leg (vacuum)

# MPS of the GS
A = Array{Any}(undef,1,N);
# Array{Any} - creates an array that can hold any type of data.
# (undef,1,N) - creates a 1D array of N elements currently holding undefined values.

# initialization
Hnow = H0
D,V = eigen((Hnow+Hnow')/2)
A[1] = contract(A0,3,3,V,2,1)
Hprev = diagm(D)

for itN in (2:N)

    global A, Hprev;

    # step [i-ii]
    # spin operator at the current site; to be used for generating
    # the coupling term at the next iteration
    Sprev = updateLeft([],[],A[itN-1],LSpace,3,A[itN-1])
    
    # step [iii]
    # # add new site
    Anow = getIdentity(Hprev,2,Id,2)
    Hnow = updateLeft(Hprev,2,Anow,[],[],Anow)
    # update the Hamiltonian up to the last sites
    # to the enlarged Hilbert space
    
    # step [iv]
    # # spin-spin interaction
    Sprev_x = (Sprev[:,1,:]+Sprev[:,2,:])/sqrt(2); Rx = size(Sprev_x);
    Sprev_y = (Sprev[:,1,:]-Sprev[:,2,:])/sqrt(2)/1im; Ry = size(Sprev_y);
    if length(Rx) == 2; Sprev_x = reshape(Sprev_x,(Rx[1],1,Rx[2])); end;
    if length(Ry) == 2; Sprev_y = reshape(Sprev_y,(Ry[1],1,Ry[2])); end;
    Sx = reshape((LSpace[:,1,:]+LSpace[:,2,:])/sqrt(2),(2,1,2));
    Sy = reshape((LSpace[:,1,:]-LSpace[:,2,:])/sqrt(2)/1im,(2,1,2));
    Hxy = - updateLeft(Sprev_x,3,Anow,Sx,3,Anow) - updateLeft(Sprev_y,3,Anow,Sy,3,Anow)
    
    # step [v]
    Hnow = Hnow+Hxy
    
    # diagonalize the current Hamiltonian
    D,V = eigen((Hnow+Hnow')/2)
    # sort eigenvalues & eigenvectors in the order of increasing
    # eigenvalues
    ids = sortperm(D)
    D = D[ids]
    V = V[:,ids]
    
    # truncation threshold for energy
    Etr = D[min(length(D),Nkeep)]
    oks = (D .< Etr)
    # true: to keep; false: not to keep
    # keep all degenerate states up to tolerance
    
    if itN < N
        A[itN] = contract(Anow,3,3,V[:,oks],2,1)
    else
        A[itN] = contract(Anow,3,3,reshape(V[:,1],(size(V[:,1])[1],1)),2,1) # select GS at the last step
    end
    Hprev = diagm(D[oks])   
end

In [32]:
for itN in (length(A):-1:1)

    T = A[itN]
    #print(1)
    T = reshape(T,(size(T,1),size(T,2)*size(T,3)))
    SVDT = svd(T)

    #print(2)

    A[itN] = reshape(SVDT.Vt,(size(SVDT.V,2),size(A[itN],2),convert(Int,size(SVDT.V,1)/size(A[itN],2))))

    #print(3)

    if itN>1
        A[itN-1] = contract(A[itN-1],3,3,SVDT.U*diagm(SVDT.S),2,1)
    else 
        A[itN] = contract(SVDT.U*diagm(SVDT.S),2,1,A[itN],3,1) # I  don't know why index of A contracting at this step be 1, I think it is because of the contraction at the previous step when itN=2.
    end
end



In [34]:
Sent_A = zeros(N-1,1); # entanglement entropy for rhoA
Sent_B = zeros(N-1,1); # entanglement entropy for rhoB
TrRhoA = zeros(N-1,1); # trace of rhoA
TrRhoB = zeros(N-1,1); # trace of rhoB

for it in (1:N-1)
    
    # # compute entanglement entropy for rhoA & trace of rhoA # #
    rho_A = contract(A[it],3,3,conj(A[it]),3,3)
    rho_A = reshape(rho_A,(size(rho_A,1)*size(rho_A,2),size(rho_A,3)*size(rho_A,4)))
    w,_ = eigen((rho_A+rho_A'))
    w = w[w .> 0] ./ 2
    Sent_A[it] = sum(-w.*log.(w))
    TrRhoA[it] = sum(w)
    # # # #

    # # compute entanglement entropy for rhoB & trace of rhoB # #
    rho_B = contract(A[it],3,[1,2],conj(A[it]),3,[1,2])
    w,_ = eigen((rho_B+rho_B'))
    w = w[w .> 0] ./ 2
    Sent_B[it] = sum(-w.*log.(w))
    TrRhoB[it] = sum(w)
    # # # #
    
    # reshape M[it] & SVD
    T = A[it]
    T = reshape(T,(size(T,1)*size(T,2),size(T,3)))
    SVDT = svd(T)
    # reshape U into rank-3 tensor, & replace M[it] with it
    A[it] = reshape(SVDT.U,(convert(Int,size(SVDT.U,1)/size(A[it],2)),
            size(A[it],2),size(SVDT.U,2)))
    if it < N
        # contract S & V' with M[it+1]
        A[it+1] = contract(diagm(SVDT.S)*SVDT.Vt,2,2,A[it+1],3,1)
    else
        global R1
        # R1: tensor which is the leftover after transforming the left
        # part. It will be SVD-ed & its left/right singular vectors 
        # will be contracted with the neighbouring M-tensors.
        R1 = SVDT.S*SVDT.Vt
    end
end
SVDR1 = svd(R1)
# V' is a single number which serves as the overall phase factor to the MPS.
# So we can pass over V' to U.
A[end] = contract(A[end],3,3,SVDR1.U*SVDR1.Vt,2,1);

UndefVarError: UndefVarError: `R1` not defined